In [46]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import xlrd
import pathlib
from pathlib import Path
import re

### Skapa en Data Frame utifrån excelfiler i en Mapp + ta bort duplikater

In [47]:
### vilken mapp vill du kolla i?
mapp = 'peptide_sample_inf'
## Skapar en lista över excelfilerna i mappen
files = os.listdir(mapp)
files_xls = [f for f in files if f[-3:] == 'lsx']
files_xls
## Läser in filerna 1 och 1 och appendar dom i df
df = pd.DataFrame()
for f in files_xls:
    data = pd.read_excel(mapp + '/' + f)
    ## sortera bort om feature<4
    data = data.loc[(data['#Feature'] >= 4)]
    ## ta bort duplikater
    data['Peptide'] = data['Peptide'].str.replace('[^a-zA-Z]', '')
    data= data.groupby(by=['Peptide'], as_index=False).sum()
    df = df.append(data)
df.reset_index(inplace=True, drop=True)
df

,Peptide,-10lgP,Mass,Length,ppm,m/z,RT,Area Sample 31,Fraction,#Feature,#Feature Sample 31,Area Sample 34,#Feature Sample 34,Area Sample 39,#Feature Sample 39
0,AAHLPAEF,28.23,854.4286,8,1.0,428.2220,73.72,7410000.0,11,4,4.0,NaN,NaN,NaN,NaN
1,AAHLPAEFTPA,50.35,1123.5662,11,0.0,562.7903,79.37,444000000.0,9,4,4.0,NaN,NaN,NaN,NaN
2,AAHLPAEFTPAV,49.91,1222.6345,12,3.5,612.3267,93.08,350000000.0,9,4,4.0,NaN,NaN,NaN,NaN
3,AAHLPAEFTPAVH,61.71,1359.6935,13,1.5,454.2391,76.47,752000000.0,9,7,7.0,NaN,NaN,NaN,NaN
4,AAHLPAEFTPAVHA,55.47,1430.7306,14,1.6,477.9182,80.96,253000000.0,9,8,8.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2131,YQKVVAGV,38.88,862.4912,8,0.9,432.2533,47.12,NaN,57,5,NaN,NaN,NaN,18200000.0,5.0
2132,YQKVVAGVAN,40.93,1047.5713,10,3.6,524.7948,43.03,NaN,57,4,NaN,NaN,NaN,5510000.0,4.0
2133,YQKVVAGVANA,48.40,1118.6084,11,4.9,560.3142,48.82,NaN,58,4,NaN,NaN,NaN,4020000.0,4.0
2134,YQKVVAGVANAL,48.54,1231.6924,12,4.6,616.8563,73.70,NaN,57,4,NaN,NaN,NaN,11200000.0,4.0


### Formaterar Data Framen utifrån vilken fil den skapats utifrån

In [48]:
## Vad för typ av fil hanterar du? (vilka kolummner önskas ha kvar)
peptide = ['Peptide', '#Feature']
protein = ['Protein Accession', 'Peptide', 'Mass', 'Length', '#Feature', '-10lgP', 'Start', 'End']
## Formatera dataframen ändra [] för att byta formatering
df = df[peptide]
## Skapa column "count"
df.insert(2,'count', 1)
df

,Peptide,#Feature,count
0,AAHLPAEF,4,1
1,AAHLPAEFTPA,4,1
2,AAHLPAEFTPAV,4,1
3,AAHLPAEFTPAVH,7,1
4,AAHLPAEFTPAVHA,8,1
...,...,...,...
2131,YQKVVAGV,5,1
2132,YQKVVAGVAN,4,1
2133,YQKVVAGVANA,4,1
2134,YQKVVAGVANAL,4,1


In [49]:
df= df.groupby(by=['Peptide'], as_index=False).mean()
#df.sort_values('Peptide', ascending=False, inplace=True)
#df['#Feature'].to_clipboard()
df

,Peptide,#Feature,count
0,AAHLPAEF,4.000000,1.0
1,AAHLPAEFTPA,4.000000,1.0
2,AAHLPAEFTPAV,4.000000,1.0
3,AAHLPAEFTPAVH,6.333333,1.0
4,AAHLPAEFTPAVHA,7.666667,1.0
...,...,...,...
1429,YQKVVAGVAN,4.000000,1.0
1430,YQKVVAGVANA,4.000000,1.0
1431,YQKVVAGVANAL,4.000000,1.0
1432,YQKVVAGVANALA,4.000000,1.0


In [60]:
df_amp= pd.DataFrame()
df_amp['Peptide'] = df['Peptide']
df_amp.reset_index(drop=True, inplace=True)
df_amp.reset_index(inplace=True)
df_amp['index'] = '>' + df_amp['index'].astype(str)
df_amp['fasta'] = df_amp['index'] + os.linesep + df_amp['Peptide']
df_amp = df_amp['fasta']
#df_amp = df_amp.set_index('index')
df_amp.to_excel('amp-inf.xlsx')

### HÄR BÖRJAR NON-inf

In [61]:
### vilken mapp vill du kolla i?
mapp = 'peptide_sample_ninf'
## Skapar en lista över excelfilerna i mappen
files = os.listdir(mapp)
files_xls = [f for f in files if f[-3:] == 'lsx']
files_xls
## Läser in filerna 1 och 1 och appendar dom i df
df = pd.DataFrame()
for f in files_xls:
    data = pd.read_excel(mapp + '/' + f)
    ## sortera bort om feature<4
    data = data.loc[(data['#Feature'] >= 4)]
    ## ta bort duplikater
    data['Peptide'] = data['Peptide'].str.replace('[^a-zA-Z]', '')
    data= data.groupby(by=['Peptide'], as_index=False).sum()
    df = df.append(data)
df.reset_index(inplace=True, drop=True)
df

,Peptide,-10lgP,Mass,Length,ppm,m/z,RT,Area Sample 13,Fraction,#Feature,#Feature Sample 13,Area Sample 21,#Feature Sample 21,Area Sample 33,#Feature Sample 33
0,AAHLPAEFTPAVHA,46.78,1430.7306,14,0.3,477.9176,85.08,1690000.0,3,4,4.0,NaN,NaN,NaN,NaN
1,AAWGKVGAHAGEYGAEALE,36.59,1885.8958,19,0.4,629.6395,85.89,13000000.0,3,4,4.0,NaN,NaN,NaN,NaN
2,AAWGKVGAHAGEYGAEALER,56.47,2041.9969,20,2.6,681.6747,78.48,10700000.0,1,7,7.0,NaN,NaN,NaN,NaN
3,ADALTNAVAHV,38.91,1080.5564,11,0.2,541.2856,79.63,2110000.0,1,4,4.0,NaN,NaN,NaN,NaN
4,ADSGEGDFLAEGGGVR,58.48,1535.6852,16,0.3,768.8501,75.14,62900000.0,2,9,9.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1448,YGTGSETESPR,46.05,1182.5153,11,0.1,592.2650,32.26,NaN,35,4,NaN,NaN,NaN,686000.0,4.0
1449,YGTGSETESPRNPS,46.84,1480.6429,14,2.0,741.3302,36.67,NaN,33,4,NaN,NaN,NaN,502000.0,4.0
1450,YLPAVDEKL,34.10,1046.5648,9,1.0,524.2902,75.79,NaN,33,4,NaN,NaN,NaN,592000.0,4.0
1451,YLYEIAR,31.57,926.4861,7,-0.1,464.2503,74.65,NaN,35,4,NaN,NaN,NaN,21700000.0,4.0


### Formatera

In [62]:
## Vad för typ av fil hanterar du? (vilka kolummner önskas ha kvar)
peptide = ['Peptide', '#Feature']
protein = ['Protein Accession', 'Peptide', 'Mass', 'Length', '#Feature', '-10lgP', 'Start', 'End']
## Formatera dataframen ändra [] för att byta formatering
df = df[peptide]
## Skapa column "count"
df.insert(2,'count', 1)
## göra mean på kopior
df= df.groupby(by=['Peptide'], as_index=False).mean()
df

,Peptide,#Feature,count
0,AAHLPAEFTPA,4.0,1.0
1,AAHLPAEFTPAV,4.0,1.0
2,AAHLPAEFTPAVH,4.0,1.0
3,AAHLPAEFTPAVHA,5.0,1.0
4,AAHLPAEFTPAVHASLDKF,6.0,1.0
...,...,...,...
1031,YLPAVDEKL,4.0,1.0
1032,YLYEIAR,4.0,1.0
1033,YPWTQRF,6.0,1.0
1034,YQKVVAGVANALAHKYH,4.0,1.0


### Skapa Fasta filen

In [63]:
df_amp= pd.DataFrame()
df_amp['Peptide'] = df['Peptide']
df_amp.reset_index(drop=True, inplace=True)
df_amp.reset_index(inplace=True)
df_amp['index'] = '>' + df_amp['index'].astype(str)
df_amp['fasta'] = df_amp['index'] + os.linesep + df_amp['Peptide']
df_amp = df_amp['fasta']
#df_amp = df_amp.set_index('index')
df_amp.to_excel('amp-ninf.xlsx')

### Här börjar Wound fluid!!!

In [65]:
### vilken mapp vill du kolla i?
mapp = 'WF'
## Skapar en lista över excelfilerna i mappen
files = os.listdir(mapp)
files_xls = [f for f in files if f[-3:] == 'lsx']
files_xls
## Läser in filerna 1 och 1 och appendar dom i df
df = pd.DataFrame()
for f in files_xls:
    data = pd.read_excel(mapp + '/' + f)
    ## sortera bort om feature<4
    data = data.loc[(data['#Feature'] >= 4)]
    ## ta bort duplikater
    data['Peptide'] = data['Peptide'].str.replace('[^a-zA-Z]', '')
    data= data.groupby(by=['Peptide'], as_index=False).sum()
    df = df.append(data)
df.reset_index(inplace=True, drop=True)
df

,Peptide,-10lgP,Mass,Length,ppm,m/z,RT,Area WF1,Fraction,#Feature,#Feature WF1,Area WF2,#Feature WF2,Area WF 3,#Feature WF 3,Area WF4,#Feature WF4,Area WF5,#Feature WF5
0,AAGSDVDIDMAFILDSAETT,61.04,2056.9146,20,4.6,1029.4692,109.31,892000.0,4,4,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AALLSPYSYSTTAVVTNPKE,85.56,2111.0786,20,2.0,1056.5487,75.48,70100000.0,3,8,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AAMSTYTGIFTDQVLSVLKGEE,85.25,2375.1565,22,2.2,1188.5881,113.66,42600000.0,4,7,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AASQAALGL,49.02,800.4392,9,0.0,401.2269,37.79,18300000.0,3,4,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AAWIDSVLA,47.32,944.4967,9,0.7,473.2560,106.09,701000.0,3,4,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4058,YPWTQRF,45.51,996.4818,7,1.6,499.2490,55.28,NaN,18,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22500000.0,4.0
4059,YQGAIRQIDDIDV,51.08,1504.7521,13,5.5,753.3875,63.75,NaN,20,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1850000.0,4.0
4060,YQKSTELLI,41.45,1093.6019,9,3.0,547.8099,38.71,NaN,18,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1800000.0,4.0
4061,YQKVVAGVANALAHKYH,58.64,1868.0056,17,1.8,623.6769,30.13,NaN,19,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58100000.0,5.0


In [66]:
## Vad för typ av fil hanterar du? (vilka kolummner önskas ha kvar)
peptide = ['Peptide', '#Feature']
protein = ['Protein Accession', 'Peptide', 'Mass', 'Length', '#Feature', '-10lgP', 'Start', 'End']
## Formatera dataframen ändra [] för att byta formatering
df = df[peptide]
## Skapa column "count"
df.insert(2,'count', 1)
## göra mean på kopior
df= df.groupby(by=['Peptide'], as_index=False).mean()
df

,Peptide,#Feature,count
0,AAFFDTASTGKTFPGFFSPML,5.00,1.0
1,AAGSDVDIDMAFILDSAETT,4.00,1.0
2,AALLSPYSYSTTAVVTNPKE,7.20,1.0
3,AAMSTYTGIFTDQVLSVLKGEE,6.75,1.0
4,AASQAALGL,4.00,1.0
...,...,...,...
1964,YVNKEIQNAV,4.00,1.0
1965,YVNKEIQNAVNGV,4.00,1.0
1966,YVNKEIQNAVNGVKQI,11.00,1.0
1967,YYLQGAKIPKPEA,5.00,1.0


In [67]:
df_amp= pd.DataFrame()
df_amp['Peptide'] = df['Peptide']
df_amp.reset_index(drop=True, inplace=True)
df_amp.reset_index(inplace=True)
df_amp['index'] = '>' + df_amp['index'].astype(str)
df_amp['fasta'] = df_amp['index'] + os.linesep + df_amp['Peptide']
df_amp = df_amp['fasta']
#df_amp = df_amp.set_index('index')
df_amp.to_excel('amp-WF.xlsx')